In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import nltk
import pandas as pd
import re
from PIL import Image, ImageFilter
from PIL import Image, ImageEnhance
import os

In [ ]:
# !pip install datefinder

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

,image_path,date,amount


In [16]:
# Create an empty DataFrame to store the data
data = {'image_path': [], 'date': [], 'amount': []}
df = pd.DataFrame(data)
df

# Replace 'your_folder_path' with the path to your folder containing images
folder_path = '../Data/Internet_bills_dialog/Images'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter the list to keep only image files (you can add more extensions if needed)
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
image_files = [f for f in file_list if any(f.lower().endswith(ext) for ext in image_extensions)]

# Loop through the image files and open each one
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)
    
    # Open the image using PIL (Python Imaging Library)
    try:
        image = Image.open(image_path)
        print(image_path)

        # image.show()


        image = image.filter(ImageFilter.EDGE_ENHANCE)

                # Enhance the brightness, contrast, and saturation
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.2)  # Increase brightness by 20%

        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.5)  # Increase contrast by 20%

        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.2)  # Increase saturation by 20%

        # show the image
        # image.show()

        # image = image.filter(ImageFilter.CONTOUR)


        # image.show()


        width, height = image.size


        logo_box_coordinates = (44, 47, 196, 140)  
        date_box_coordinates = (27, 284, 231, 517)
        amount_box_coordinates = (296, 300, 772, 555) 

        logo_cropped_image = image.crop(logo_box_coordinates)
        # logo_cropped_image = logo_cropped_image.filter(ImageFilter.EDGE_ENHANCE)
    

        date_cropped_image = image.crop(date_box_coordinates)
        # date_cropped_image = date_cropped_image.filter(ImageFilter.EDGE_ENHANCE)


        amount_cropped_image = image.crop(amount_box_coordinates)
        # amount_cropped_image = amount_cropped_image.filter(ImageFilter.EDGE_ENHANCE)


        # date_cropped_image.show()
        # amount_cropped_image.show()

        text_all = pytesseract.image_to_string(image)
        text_logo = pytesseract.image_to_string(date_cropped_image)
        text_date = pytesseract.image_to_string(date_cropped_image)
        text_amount = pytesseract.image_to_string(amount_cropped_image)





        # replace | and [ with space
        # text_logo = text_logo.replace('|', '')
        # text_logo = text_logo.replace('[', '')

        text_date = text_date.replace('|', '')
        text_date = text_date.replace('[', '')

        text_amount = text_amount.replace('|', '')
        text_amount = text_amount.replace('[', '')

        # print(text_date)
        # print(text_amount)

        # print("========================================================================================================")


        # Define a regular expression pattern to match the billing date
        billing_date_pattern = r'Date\s+(\d{4}-\d{2}-\d{2})'

        # Search for the billing date using the pattern
        billing_date_match = re.search(billing_date_pattern, text_date)

        # Extract and print the billing date if found
        if billing_date_match:
            billing_date = billing_date_match.group(1)
            # print(f"Billing Date: {billing_date}")
        else:
            print("Date not found: " , text_date)



        # Define a regular expression pattern to match the total charges for the period
        total_charges_pattern = r'Total\s+([\d,.]+)'

        # Search for the total charges using the pattern
        total_charges_match = re.search(total_charges_pattern, text_amount)

        # Extract and print the total charges if found
        if total_charges_match:
            total_charges = total_charges_match.group(1)
            # Remove commas and convert to float
            total_charges = float(total_charges.replace(',', ''))
            # print(f"Total Charges for the Period: {total_charges:.2f}")
        else:
            print("Total not found: " , text_amount)

        # print("all:" , text_all)


        df = df.append({'image_path': image_path, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


        print("========================================================================================================")
    
    except Exception as e:
        print(f"Error opening {image_file}: {str(e)}")




../Data/Internet_bills_dialog/Images\1.png
Date not found:  Payment information
Date
2021-1248 17:04:00

Status
Success

Payment Method
Credit Card

../Data/Internet_bills_dialog/Images\10.png


C:\Users\User\AppData\Local\Temp\ipykernel_30328\2620588473.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path, 'date': billing_date, 'amount': total_charges}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_30328\2620588473.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


../Data/Internet_bills_dialog/Images\11.png


C:\Users\User\AppData\Local\Temp\ipykernel_30328\2620588473.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


../Data/Internet_bills_dialog/Images\12.png
Date not found:  Payment information

Status
Success

Payment Method
Credit Card

../Data/Internet_bills_dialog/Images\2.png


...

C:\Users\User\AppData\Local\Temp\ipykernel_30328\2620588473.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


../Data/Internet_bills_dialog/Images\8.png


C:\Users\User\AppData\Local\Temp\ipykernel_30328\2620588473.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


../Data/Internet_bills_dialog/Images\9.png
Date not found:  Payment information

Status
Success

Payment Method
Credit Card



C:\Users\User\AppData\Local\Temp\ipykernel_30328\2620588473.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


In [17]:
df

,image_path,date,amount
0,../Data/Internet_bills_dialog/Images\1.png,2022-12-27,2890.44
1,../Data/Internet_bills_dialog/Images\10.png,2023-01-04,2800.00
2,../Data/Internet_bills_dialog/Images\11.png,2023-01-26,4200.00
3,../Data/Internet_bills_dialog/Images\12.png,2023-01-26,4000.00
4,../Data/Internet_bills_dialog/Images\2.png,2022-03-23,500.00
5,../Data/Internet_bills_dialog/Images\3.png,2022-05-31,800.00
6,../Data/Internet_bills_dialog/Images\4.png,2022-07-04,2226.56
7,../Data/Internet_bills_dialog/Images\5.png,2022-07-15,800.00
8,../Data/Internet_bills_dialog/Images\6.png,2022-08-07,800.00
9,../Data/Internet_bills_dialog/Images\7.png,2022-08-28,4387.23


In [18]:
# get year and month from date
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [19]:
df

,image_path,date,amount,year,month
0,../Data/Internet_bills_dialog/Images\1.png,2022-12-27,2890.44,2022,12
1,../Data/Internet_bills_dialog/Images\10.png,2023-01-04,2800.00,2023,1
2,../Data/Internet_bills_dialog/Images\11.png,2023-01-26,4200.00,2023,1
3,../Data/Internet_bills_dialog/Images\12.png,2023-01-26,4000.00,2023,1
4,../Data/Internet_bills_dialog/Images\2.png,2022-03-23,500.00,2022,3
5,../Data/Internet_bills_dialog/Images\3.png,2022-05-31,800.00,2022,5
6,../Data/Internet_bills_dialog/Images\4.png,2022-07-04,2226.56,2022,7
7,../Data/Internet_bills_dialog/Images\5.png,2022-07-15,800.00,2022,7
8,../Data/Internet_bills_dialog/Images\6.png,2022-08-07,800.00,2022,8
9,../Data/Internet_bills_dialog/Images\7.png,2022-08-28,4387.23,2022,8


In [28]:
image = Image.open('../Data/Internet_bills_dialog/Images/1.png')

image = image.filter(ImageFilter.EDGE_ENHANCE)
image.show()


        # Enhance the brightness, contrast, and saturation
enhancer = ImageEnhance.Brightness(image)
image = enhancer.enhance(1.5)  # Increase brightness by 20%
image.show()


enhancer = ImageEnhance.Contrast(image)
image = enhancer.enhance(1.5)  # Increase contrast by 20%
image.show()


enhancer = ImageEnhance.Color(image)
image = enhancer.enhance(1.2)  # Increase saturation by 20%
image.show()


# enhancer = ImageEnhance.Sharpness(image)
# image = enhancer.enhance(1.5)  # Increase sharpness by 50%
# image.show()

In [29]:
import re

# Your input text
text = "This is a sample text with myDialog, SLT, and Hutch mentioned."

# Initialize the isp variable to None
isp = None

# Check for "Dialog" or "myDialog"
dialog_pattern = r'\b(?:my)?dialog\b'
if re.search(dialog_pattern, text, re.IGNORECASE):
    isp = "Dialog"

# Check for "SLT" or "Sri Lanka Telecom"
slt_pattern = r'\bslt\b|\bsri\s+lanka\s+telecom\b'
if re.search(slt_pattern, text, re.IGNORECASE):
    isp = "SLT"

# Check for "Hutch"
hutch_pattern = r'\bhutch\b'
if re.search(hutch_pattern, text, re.IGNORECASE):
    isp = "Hutch"

# Check if any condition was matched
if isp is not None:
    print(f"The ISP is: {isp}")
else:
    print("No specific ISP detected.")


The text contains 'Dialog' or 'myDialog'.
